# Improving Our VAE
*Kiya Aminfar, Sean Steinle*

This notebook is our attempt to improve upon `./simple_vae.ipynb`, where we trained a simple but ineffective variational autoencoder to generate music. In this notebook, we try three techniques to improve our autoencoder: more epochs, more data, and more layers.

## Table of Contents
1. [More Epochs](#More-Epochs)
2. [More Data](#More-Data)
3. [More Layers](#More-Layers)

In [1]:
import librosa, os #audio processing and file system parsing
import librosa.display
import numpy as np #math library
import tensorflow as tf #for model building
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt # for visualization
import pandas as pd #for data analysis / prep
import IPython.display as ipd #for sound output

2025-03-14 14:22:49.813725: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 14:22:49.840199: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 14:22:50.087594: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 14:22:50.261332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741976570.410230   26290 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741976570.45

## More Epochs

## More Data

It looks like the [Free Music Archive](https://github.com/mdeff/fma) is the easiest way to get access to more music! They have a paper detailing all of their work, freely available metadata, and audio-only zips. Let's start with the smallest audio-only zip (`fma_small.zip`), which contains about 8,000 tracks!

In [2]:
# Parameters
SAMPLE_RATE = 22050  # Standard sample rate for music processing
N_MELS = 128         # Number of Mel filterbanks
HOP_LENGTH = 512     # Hop length for STFT
N_FFT = 2048         # FFT window size
DURATION = 5         # Duration of each audio clip in seconds
BATCH_SIZE = 32

def load_audio_to_mel(file_path):
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convert to log scale (dB)
    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min())
    return mel_spec_norm

def mel_to_audio(mel_spec, sr=22050, n_fft=2048, hop_length=512, n_mels=128, power=1.0):
    """
    Convert a Mel spectrogram back to audio using the Griffin-Lim algorithm.
    Args:
        mel_spec: Mel spectrogram (shape: [n_mels, time_steps])
        sr: Sample rate for the audio
        n_fft: FFT size for Griffin-Lim
        hop_length: Hop length for Griffin-Lim
        n_mels: Number of Mel bins in the spectrogram
        power: Exponent for the spectrogram
    Returns:
        Audio signal as a numpy array
    """
    # Invert Mel to linear scale
    mel_inverted = librosa.feature.inverse.mel_to_audio(mel_spec ** power, sr=sr, n_fft=n_fft, hop_length=hop_length)
    return mel_inverted

def load_df_from_kaggle(music_dir: str):
    """This function loads a directory of songs and creates a simple dataframe with (genre,song,numpy) rows, based on the Kaggle dataset. We expect music_dir to be a 2-level directory with genre directories on the first level and .wav songs on the seocnd level."""
    music_dicts,bad_paths = [],[]
    genres = os.listdir(music_dir)
    for genre in genres:
        try:
            for song in os.listdir(music_dir+genre):
                song_path = music_dir+genre+'/'+song
                try:
                    music_dicts.append({'genre': genre, 'song': song, 'numpy_representation': load_audio_to_mel(song_path)})
                except Exception as e:
                    print(f"couldn't load: {song_path}, got: {e}")
                    bad_paths.append([song_path,e])
        except Exception as e:
            print(f"couldn't process the {genre} directory, got: {e}")
    return pd.DataFrame(music_dicts)

def load_df_from_fma(music_dir: str, max_songs: int=8000):
    """This function loads a directory of songs and creates a simple dataframe with (song,numpy) rows, based on the FMA dataset. We expect music_dir to be a single-level directory with just songs."""
    music_dicts,bad_paths = [],[]
    for i,song in enumerate(os.listdir(music_dir)):
        song_path = music_dir+song
        try:
            audio = load_audio_to_mel(song_path)
            if np.isnan(audio).sum() > 0: 
                raise Exception("got nan in audio")
            music_dicts.append({'song': song, 'numpy_representation': audio})
        except Exception as e:
            print(f"couldn't load: {song_path}, got: {e}")
            bad_paths.append([song_path,e])
        if i >= max_songs:
            break
    return pd.DataFrame(music_dicts)

def create_tf_dataset(music_df):
    """Converts music_df's numpy column to a TensorFlow dataset."""
    assert "numpy_representation" in music_df.columns
    numpy_representations = np.array(music_df["numpy_representation"].tolist(), dtype=np.float32)

    # Ensure shape is (128, 216, 1) (Mel bins, Time steps, Channels)
    numpy_representations = np.expand_dims(numpy_representations, -1)  # Add channel dimension

    songs_dataset = tf.data.Dataset.from_tensor_slices(numpy_representations)
    songs_dataset = songs_dataset.shuffle(len(numpy_representations)).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    return songs_dataset

In [ ]:
music_df = load_df_from_kaggle("../data/genres_original/")
songs_dataset = create_tf_dataset(music_df)

## More Layers

In [ ]:
# Parameters
LATENT_DIM = 64  # Latent space dimension
INPUT_SHAPE = (128, 216)  # (Time steps, Mel bins)

# Reparameterization Trick
def sampling(args):
    """Reparameterization trick: z = mu + exp(log_var / 2) * epsilon"""
    mu, log_var = args
    epsilon = tf.keras.backend.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var * 0.5) * epsilon

# Encoder
def build_encoder(input_shape, latent_dim):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(32, kernel_size=3, strides=2, padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, kernel_size=3, strides=2, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, kernel_size=3, strides=2, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    
    mu = layers.Dense(latent_dim, name="latent_mu")(x)
    log_var = layers.Dense(latent_dim, name="latent_log_var")(x)
    z = layers.Lambda(sampling, name="latent_sample")([mu, log_var])
    
    return Model(inputs, [mu, log_var, z], name="Encoder")

# Decoder
def build_decoder(latent_dim, output_shape):
    decoder_inputs = layers.Input(shape=(latent_dim,))
    
    x = layers.Dense(output_shape[0] // 8 * 128, activation="relu")(decoder_inputs)
    x = layers.Reshape((output_shape[0] // 8, 128))(x)  # Adjust size
    
    x = layers.Conv1DTranspose(128, kernel_size=3, strides=2, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1DTranspose(64, kernel_size=3, strides=2, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1DTranspose(32, kernel_size=3, strides=2, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    
    outputs = layers.Conv1DTranspose(output_shape[1], kernel_size=3, padding="same", activation="sigmoid")(x)
    return Model(decoder_inputs, outputs, name="Decoder")

# VAE Model
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def compute_loss(self, inputs):
        """Computes the VAE loss function."""
        mu, log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)

        # Reconstruction loss
        recon_loss = tf.reduce_mean(tf.keras.losses.mse(inputs, reconstructed))

        # KL Divergence loss
        kl_loss = -0.5 * tf.reduce_mean(1 + log_var - tf.square(mu) - tf.exp(log_var))

        return recon_loss + kl_loss

    def train_step(self, data):
        """Custom training step."""
        with tf.GradientTape() as tape:
            loss = self.compute_loss(data)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return {"loss": loss}

def generate_samples(model, num_samples=10):
    """Convenience function for generating num_samples from model."""
    random_latent_vectors = tf.random.normal(shape=(num_samples, LATENT_DIM)) # Sample random latent vectors from the prior (e.g., normal distribution)
    generated_samples = model.decoder(random_latent_vectors) # Use the decoder to generate samples from the random latent vectors
    generated_samples = [s.numpy().squeeze() for s in generated_samples] #convert to numpy, reshape
    return generated_samples

In [ ]:
songs_dataset

In [ ]:
INPUT_SHAPE

In [ ]:
# Build the encoder and decoder
encoder = build_encoder(INPUT_SHAPE, LATENT_DIM)
decoder = build_decoder(LATENT_DIM, INPUT_SHAPE)

# Create the VAE instance
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())

# Train the model
n_epochs = 20
vae.fit(songs_dataset, epochs=n_epochs)

In [ ]:
generated_samples = generate_samples(vae, num_samples=5)
mel_audio = mel_to_audio(generated_samples[0])
ipd.Audio(mel_audio, rate=22050) #generated sample

In [ ]:
#hm, same poor results. what if we just train with classical and if we train for longer?

In [ ]:
music_df = load_df_from_kaggle("../data/genres_original/")
classical_df = music_df[music_df['genre'] == 'classical']
songs_dataset = create_tf_dataset(classical_df)

In [ ]:
# Build the encoder and decoder
encoder = build_encoder(INPUT_SHAPE, LATENT_DIM)
decoder = build_decoder(LATENT_DIM, INPUT_SHAPE)

# Create the VAE instance
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())

# Train the model
n_epochs = 100
vae.fit(songs_dataset, epochs=n_epochs)

In [ ]:
generated_samples = generate_samples(vae, num_samples=5)
mel_audio = mel_to_audio(generated_samples[0])
ipd.Audio(mel_audio, rate=22050) #generated sample

## The FMA

In [ ]:
mp3_track = "../data/fma/small/000002.mp3"
audio, sr = librosa.load(mp3_track, sr=None)  # sr=None keeps the original sample rate
print("Original Ground-Truth Audio:")
ipd.display(ipd.Audio(audio, rate=sr))

In [ ]:
load_audio_to_mel(mp3_track)

In [ ]:
fma_df = load_df_from_fma("../data/fma/small/")
songs_dataset = create_tf_dataset(fma_df)
fma_df

In [ ]:
# Build the encoder and decoder
encoder = build_encoder(INPUT_SHAPE, LATENT_DIM)
decoder = build_decoder(LATENT_DIM, INPUT_SHAPE)

# Create the VAE instance
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())

# Train the model
n_epochs = 5
vae.fit(songs_dataset, epochs=n_epochs)

In [ ]:
generated_samples = generate_samples(vae, num_samples=5)
mel_audio = mel_to_audio(generated_samples[0])
ipd.Audio(mel_audio, rate=22050) #generated sample